In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('IPL Ball-by-Ball 2008-2020.csv')

In [3]:
df = pd.read_csv('IPL Matches 2008-2020.csv')

## Preprocessing

In [4]:
df.replace({'Rising Pune Supergiant': 'Rising Pune Supergiants','Delhi Daredevils':'Delhi Capitals', 'Deccan Chargers':'Sunrisers Hyderabad', 'Pune Warriors':'Rising Pune Supergiants'},inplace=True)
df.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [5]:
df["city"].fillna("Dubai", inplace = True)

# Batsman KPIs

1. Strike rate
2. Percentage of not out
3. Percentage of dot balls
4. Percentage of boundaries
5. Man of the match count
6. Percentage of 30+ runs scored in matches
7.average runs per match
8.strike rate in power play and deth over

<b>Calculating Total Runs</b>

In [6]:
no_of_runs=pd.DataFrame(data.groupby('batsman')['batsman_runs'].agg(sum)).reset_index().sort_values(by='batsman_runs',ascending=False).reset_index(drop=True)
no_of_runs.head(5)

,batsman,batsman_runs
0,V Kohli,5878
1,SK Raina,5368
2,DA Warner,5254
3,RG Sharma,5230
4,S Dhawan,5197


<b> Calculating number of ball</b>

In [7]:
no_of_balls=data.groupby('batsman')['ball'].count().reset_index().sort_values(by='ball',ascending=False).reset_index(drop=True)
no_of_balls.head(5)

,batsman,ball
0,V Kohli,4609
1,S Dhawan,4208
2,RG Sharma,4088
3,SK Raina,4041
4,DA Warner,3819


<b> Calculating number of bounderies</b>

In [8]:
boundries= data.loc[(data['batsman_runs'] == 4) | (data['batsman_runs'] == 6)]
no_of_balls_hitted_boundry=boundries.groupby('batsman')['ball'].count().reset_index().sort_values(by='ball',ascending=False).reset_index(drop=True)
no_of_balls_hitted_boundry.columns = ['batsman','boundaries']
no_of_balls_hitted_boundry.head(5)

,batsman,boundaries
0,CH Gayle,733
1,V Kohli,706
2,DA Warner,705
3,S Dhawan,700
4,SK Raina,687


<b> Calciulating number of tinmes dismissed</b>

In [9]:
wickets=data[data['is_wicket']==1]
batsman_out=wickets.groupby('batsman')['is_wicket'].count().reset_index().sort_values(by='is_wicket',ascending=False).reset_index(drop=True)
batsman_out.columns = ['batsman','dismissed']
batsman_out.head(5)

,batsman,dismissed
0,RG Sharma,177
1,RV Uthappa,169
2,V Kohli,163
3,SK Raina,160
4,S Dhawan,152


<b> Calculating total number of times dismissed</b>

In [10]:
dot_balls=data[(data['batsman_runs']==0)].groupby('batsman')['ball'].count().reset_index()
dot_balls.columns = ['batsman','dot_balls']
dot_balls.head(5)

,batsman,dot_balls
0,A Ashish Reddy,61
1,A Chandila,3
2,A Chopra,45
3,A Choudhary,4
4,A Dananjaya,2


<b> Calculating total number of times player has won man of the match</b>

In [11]:
no_man_of_the_match=df.groupby('player_of_match')['id'].count().reset_index().sort_values(by='id',ascending=False).reset_index(drop=True)
no_man_of_the_match.columns = ['batsman','man_of_match']
no_man_of_the_match.head(5)

,batsman,man_of_match
0,AB de Villiers,23
1,CH Gayle,22
2,RG Sharma,18
3,DA Warner,17
4,MS Dhoni,17


<b> Calculating total number of times scored 30+ </b>

In [12]:
total_score_in_every_match=data.groupby(['batsman','id'])['batsman_runs'].sum().reset_index()
score_above_thirty=total_score_in_every_match[total_score_in_every_match['batsman_runs']>=30].reset_index()
total_matches_above_thirty=score_above_thirty.groupby('batsman')['id'].count().reset_index()
total_matches_above_thirty.columns = ['batsman','match_30+']
total_matches_above_thirty.head(5)

,batsman,match_30+
0,A Ashish Reddy,2
1,A Mishra,2
2,A Symonds,14
3,AA Jhunjhunwala,2
4,AB Agarkar,1


<b> Total number of innings played</b>

In [13]:
total_matches_played=total_score_in_every_match.groupby('batsman')['id'].count().reset_index()
total_matches_played.columns = ['batsman','total_matches']
total_matches_played.head(5)

,batsman,total_matches
0,A Ashish Reddy,23
1,A Chandila,2
2,A Chopra,6
3,A Choudhary,3
4,A Dananjaya,1


In [14]:
#merging the data
batsmanDf = pd.merge(no_of_runs,no_of_balls,on='batsman')
batsmanDf = batsmanDf.merge(no_of_balls_hitted_boundry,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(batsman_out,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(dot_balls,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(no_man_of_the_match, left_on='batsman', right_on='batsman', how='left')
batsmanDf = batsmanDf.merge(total_matches_above_thirty, left_on='batsman', right_on='batsman', how='left')
batsmanDf = batsmanDf.merge(total_matches_played, left_on='batsman', right_on='batsman', how='left')

#filtered based on atleast 100 balls played
batsmanDf = batsmanDf[batsmanDf["ball"] >= 100]

batsmanDf.head(5)

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175


<b>Calculating Strike Rate,Percentage not out,Percentage dot balls,Percentage bounderies,Percentage 30+ runs</b>

In [15]:
batsmanDf['strike_rate']=np.round(batsmanDf['batsman_runs']/batsmanDf['ball']*100,2)
batsmanDf['%age_not_out']=np.round(100-(batsmanDf['dismissed']/batsmanDf['total_matches'])*100,2)
batsmanDf['%age_dot_balls']=np.round((batsmanDf['dot_balls']/batsmanDf['ball'])*100,2)
batsmanDf['%age_boundaries']=np.round((batsmanDf['boundaries']/batsmanDf['ball'])*100,2)
batsmanDf['%30+runs']=np.round((batsmanDf['match_30+']/batsmanDf['total_matches'])*100,2)
batsmanDf['average_suns_per_match']=np.round((batsmanDf['batsman_runs']/batsmanDf['total_matches']),2)
batsmanDf.head(5)

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches,strike_rate,%age_not_out,%age_dot_balls,%age_boundaries,%30+runs,average_suns_per_match
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184,127.53,11.41,35.26,15.32,45.65,31.95
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189,132.84,15.34,34.94,17.00,40.21,28.40
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142,137.58,11.27,37.42,18.46,53.52,37.00
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194,127.94,8.76,37.48,16.44,37.63,26.96
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175,123.50,13.14,38.33,16.63,41.14,29.70


In [16]:
powerplay_balls=data[data['over']<6].groupby('batsman').size().reset_index(name='ball')
powerplay_runs=data[data['over']<6].groupby('batsman')['batsman_runs'].agg(sum).reset_index(name='runs')
powerplay_data=pd.merge(powerplay_balls,powerplay_runs,on='batsman')
powerplay_data['Strike_rate_in_powerplay']=(powerplay_data['runs']/powerplay_data['ball'])*100
del powerplay_data['ball']
del powerplay_data['runs']
deathover_balls=data[data['over']>15].groupby('batsman').size().reset_index(name='ball')
deathover_runs=data[data['over']>15].groupby('batsman')['batsman_runs'].agg(sum).reset_index(name='batsman_runs')
deathover_data=pd.merge(deathover_balls,deathover_runs,on='batsman')
deathover_data['Strike_rate_in_death_overs']=((deathover_data['batsman_runs']/deathover_data['ball'])*100)
del deathover_data['ball']
del deathover_data['batsman_runs']
batsmanDf=pd.merge(batsmanDf,powerplay_data,left_on='batsman',right_on='batsman',how='left')
batsmanDf=pd.merge(batsmanDf,deathover_data,left_on='batsman',right_on='batsman',how='left')
batsmanDf.to_csv('batsman.csv')
batsmanDf.head(5)

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches,strike_rate,%age_not_out,%age_dot_balls,%age_boundaries,%30+runs,average_suns_per_match,Strike_rate_in_powerplay,Strike_rate_in_death_overs
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184,127.53,11.41,35.26,15.32,45.65,31.95,111.802853,197.148676
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189,132.84,15.34,34.94,17.00,40.21,28.40,129.124579,164.843750
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142,137.58,11.27,37.42,18.46,53.52,37.00,132.440783,190.721649
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194,127.94,8.76,37.48,16.44,37.63,26.96,111.317073,190.349418
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175,123.50,13.14,38.33,16.63,41.14,29.70,119.736842,158.536585


In [17]:
#Rank calculation
batsmanDf=batsmanDf.replace(np.nan,0)
batsmanDf['runs_rank']=batsmanDf.batsman_runs.rank(ascending=False)
batsmanDf['dots_rank']=batsmanDf['%age_dot_balls'].rank(ascending=True)
batsmanDf['30+_rank']=batsmanDf['match_30+'].rank(ascending=False)
batsmanDf['strike_rate_rank']=batsmanDf['strike_rate'].rank(ascending=False)
batsmanDf['not_out_rank']=batsmanDf['%age_not_out'].rank(ascending=True)
# batsmanDf['man_match_rank']=batsmanDf['man_of_match'].rank(ascending=False)
batsmanDf['batting_avg_rank']=batsmanDf['average_suns_per_match'].rank(ascending=False)
batsmanDf['powerplay_strike_rank']=batsmanDf['Strike_rate_in_powerplay'].rank(ascending=False)
batsmanDf['deathover_strike_rank']=batsmanDf['Strike_rate_in_death_overs'].rank(ascending=False)
batsmanDf['balls_played_rank']=batsmanDf['ball'].rank(ascending=False)
batsmanDf['matches_played_rank']=batsmanDf['total_matches'].rank(ascending=False)
batsmanDf['hitman_rank']=batsmanDf['%age_boundaries'].rank(ascending=False)
batsmanRank=batsmanDf[['batsman','not_out_rank','runs_rank','dots_rank','30+_rank','strike_rate_rank','batting_avg_rank','powerplay_strike_rank','deathover_strike_rank','matches_played_rank','balls_played_rank','hitman_rank']]
batsmanRank['batsman_rank']=batsmanRank[['runs_rank','30+_rank','strike_rate_rank','batting_avg_rank']].mean(axis=1)
batsmanRank['batsman_rank']=np.round(batsmanRank.batsman_rank.rank(ascending=True),0)
batsmanRank['experience_rank']=batsmanRank[['matches_played_rank','balls_played_rank']].mean(axis=1)
batsmanRank['powerplay_relibility']=batsmanRank[['powerplay_strike_rank','not_out_rank','experience_rank']].mean(axis=1)
batsmanRank['powerplay_relibility']=batsmanRank.powerplay_relibility.rank(ascending=True)
batsmanRank['deathover_relibility']=batsmanRank[['deathover_strike_rank','hitman_rank','experience_rank']].mean(axis=1)
batsmanRank['deathover_relibility']=batsmanRank.deathover_relibility.rank(ascending=True)
batsmanDf.drop(['not_out_rank','runs_rank','dots_rank','30+_rank','strike_rate_rank','batting_avg_rank','powerplay_strike_rank','deathover_strike_rank','matches_played_rank','balls_played_rank','hitman_rank'], axis=1, inplace=True)
batsmanRank=batsmanRank.sort_values(by='powerplay_relibility')
batsmanRank

C:\Users\Admin\AppData\Local\Temp/ipykernel_15080/1271793131.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsmanRank['batsman_rank']=batsmanRank[['runs_rank','30+_rank','strike_rate_rank','batting_avg_rank']].mean(axis=1)
C:\Users\Admin\AppData\Local\Temp/ipykernel_15080/1271793131.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsmanRank['batsman_rank']=np.round(batsmanRank.batsman_rank.rank(ascending=True),0)
C:\Users\Admin\AppData\Local\Temp/ipykernel_15080/1271793131.py:18: SettingWithCo

,batsman,not_out_rank,runs_rank,dots_rank,30+_rank,strike_rate_rank,batting_avg_rank,powerplay_strike_rank,deathover_strike_rank,matches_played_rank,balls_played_rank,hitman_rank,batsman_rank,experience_rank,powerplay_relibility,deathover_relibility
20,V Sehwag,33.0,21.0,114.0,24.0,12.0,41.0,15.0,6.0,23.0,29.0,4.0,8.0,26.00,1.0,1.0
17,BB McCullum,21.0,18.0,178.0,15.0,74.0,39.0,53.0,10.0,21.0,19.0,40.0,16.0,20.00,2.0,5.0
2,DA Warner,67.0,3.0,54.5,2.5,26.0,3.0,24.0,23.0,13.0,5.0,44.0,1.0,9.00,3.0,7.0
8,RV Uthappa,43.0,9.0,117.0,6.0,84.0,52.0,57.0,69.0,4.5,6.0,70.0,20.0,5.25,4.0,19.0
34,AC Gilchrist,30.5,35.0,195.0,38.5,43.0,44.5,41.0,8.0,39.0,37.0,12.0,24.0,38.00,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,R Dhawan,230.5,198.0,136.0,222.0,163.0,210.0,216.0,167.0,174.0,196.5,180.5,212.0,185.25,229.0,197.0
214,B Lee,224.0,215.0,177.0,222.0,113.0,223.0,216.0,168.5,159.5,230.0,94.5,208.0,194.75,230.0,167.0
175,Bipul Sharma,233.0,176.0,1.0,200.0,10.0,197.5,216.0,106.0,167.0,205.0,80.0,151.0,186.00,231.0,132.0
219,DL Vettori,230.5,219.5,92.5,222.0,197.0,217.0,216.0,189.0,174.0,212.0,202.0,223.0,193.00,232.0,216.5


# Bowler KPIs

1. Economy
2. Strike Rate
3. Average Wickets per match
4. Percentage of Dot balls
5. Percentage Boundaries
6. Economy for Powerplay, Middle over, Death overs
7. Bowling Average
8. 4 Wicket Haul

<b>Calculating total number of balls and overs bowled by bowler</b>

In [18]:
Bowler_over= data[data["extra_runs"]==0][["ball","bowler"]]
Bowler_over= Bowler_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_over["No_of_Overs"]=(np.floor(Bowler_over["ball"]/6))
Bowler_over.head(5)

,bowler,ball,No_of_Overs
0,Harbhajan Singh,3325,554.0
1,PP Chawla,3218,536.0
2,R Ashwin,3186,531.0
3,A Mishra,3128,521.0
4,SL Malinga,2753,458.0


<b>Calculating total runs given by bowler</b>

In [19]:
Bowler_Runs= data.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_Runs.columns=["bowler","Total_runs_given"]
Bowler_over = Bowler_over[Bowler_over["ball"] >= 60]
Bowler_Runs.head(5)

,bowler,Total_runs_given
0,PP Chawla,4330
1,Harbhajan Singh,4038
2,A Mishra,3913
3,DJ Bravo,3869
4,R Ashwin,3756


<b>Total number of bounderies given</b>

In [20]:
Bowler_boundary = data.query('batsman_runs == 4 | batsman_runs == 6')
Bowler_boundary = Bowler_boundary.groupby("bowler")["batsman_runs"].count().reset_index().sort_values(by="batsman_runs",ascending=False).reset_index(drop=True)
Bowler_boundary.columns=["bowler","Total_boundaries_given"]
Bowler_boundary.head(5)

,bowler,Total_boundaries_given
0,UT Yadav,509
1,PP Chawla,497
2,DJ Bravo,443
3,P Kumar,442
4,Harbhajan Singh,431


<b>Calculating total number of wickets given</b>

In [21]:
Bowler_wicket = data[data["dismissal_kind"] != "run out"]
Bowler_wicket= Bowler_wicket.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_wicket.columns=["bowler","Wickets"]
Bowler_wicket.head(5)

,bowler,Wickets
0,SL Malinga,170
1,A Mishra,160
2,PP Chawla,156
3,DJ Bravo,153
4,Harbhajan Singh,150


<b>Calculating total number of dot balls by bowler</b>

In [22]:
Bowler_Dots = data[data["total_runs"] == 0]
Bowler_Dots= Bowler_Dots.groupby("bowler")["total_runs"].count().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_Dots.columns=["bowler","Dot_balls"]
Bowler_Dots.head(5)

,bowler,Dot_balls
0,Harbhajan Singh,1244
1,R Ashwin,1166
2,B Kumar,1155
3,SL Malinga,1144
4,PP Chawla,1137


<b>Total number of matches bowlers played </b>

In [23]:
Bowler_matches = data.groupby("bowler")["id"].nunique().reset_index().sort_values(by="id",ascending=False).reset_index(drop=True)
Bowler_matches.columns=["bowler","no_of_match"]
Bowler_matches.head(5)

,bowler,no_of_match
0,PP Chawla,163
1,Harbhajan Singh,157
2,RA Jadeja,155
3,R Ashwin,151
4,A Mishra,150


<b>Calculating Power play overs,economy and runs given</b>

In [24]:
Bowler_powerplay_wickets = data[data["over"] < 6]
Bowler_powerplay_wickets =  Bowler_powerplay_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_powerplay_wickets.columns=["bowler","Powerplay_runs"]

Bowler_powerplay_runs = data.query('dismissal_kind != "run out" & over < 6')
Bowler_powerplay_runs= Bowler_powerplay_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_powerplay_runs.columns=["bowler","Powerplay_wickets"]

Bowler_powerplay_over = data.query('extra_runs == 0 & over < 6')
Bowler_powerplay_over= Bowler_powerplay_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_powerplay_over["Powerplay_over"]=np.round(Bowler_powerplay_over["ball"]/6,1)

Bowler_powerplay = pd.merge(Bowler_powerplay_runs,Bowler_powerplay_wickets,on="bowler")
Bowler_powerplay = pd.merge(Bowler_powerplay, Bowler_powerplay_over, on="bowler")
Bowler_powerplay["Powerplay_economy"] = np.round(Bowler_powerplay["Powerplay_runs"] / Bowler_powerplay["Powerplay_over"],1)

del Bowler_powerplay['ball']
Bowler_powerplay.head(5)

,bowler,Powerplay_wickets,Powerplay_runs,Powerplay_over,Powerplay_economy
0,Z Khan,53,1492,209.7,7.1
1,Sandeep Sharma,53,1447,201.2,7.2
2,B Kumar,48,1472,234.0,6.3
3,UT Yadav,45,1424,174.3,8.2
4,DS Kulkarni,44,1274,165.8,7.7


<b>Calculating Power play overs,economy and runs given</b>

In [25]:
Bowler_middle_wickets = data.query('over >= 6 & over <= 15')
Bowler_middle_wickets =  Bowler_middle_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_middle_wickets.columns=["bowler","Middle_runs"]

Bowler_middle_runs = data.query('dismissal_kind != "run out" & over >= 6 & over <= 15')
Bowler_middle_runs= Bowler_middle_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_middle_runs.columns=["bowler","Middle_wickets"]

Bowler_middle_over = data.query('extra_runs == 0 & over >= 6 & over <= 15')
Bowler_middle_over= Bowler_middle_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_middle_over["Middle_over"]=np.round(Bowler_middle_over["ball"]/6,1)

Bowler_middle = pd.merge(Bowler_middle_runs,Bowler_middle_wickets,on="bowler")
Bowler_middle = pd.merge(Bowler_middle, Bowler_middle_over,on="bowler")
Bowler_middle["Middle_economy"] = np.round(Bowler_middle["Middle_runs"] / Bowler_middle["Middle_over"],1)

del Bowler_middle['ball']
Bowler_middle.head(5)

,bowler,Middle_wickets,Middle_runs,Middle_over,Middle_economy
0,A Mishra,130,3319,457.3,7.3
1,PP Chawla,112,3463,436.2,7.9
2,Harbhajan Singh,101,2904,404.0,7.2
3,YS Chahal,94,2065,271.2,7.6
4,RA Jadeja,84,2854,371.3,7.7


<b>Calculating Death overs,economy and runs given</b>

In [26]:
Bowler_death_runs = data.query('over >= 16')
Bowler_death_runs =  Bowler_death_runs.groupby("bowler")["total_runs"].agg(sum).reset_index()
Bowler_death_runs.columns=["bowler","Death_runs"]

Bowler_death_wickets = data.query('dismissal_kind != "run out" & over >= 16')
Bowler_death_wickets= Bowler_death_wickets.groupby("bowler")["player_dismissed"].agg(sum).reset_index()
Bowler_death_wickets.columns=["bowler","Death_wickets"]

Bowler_death_over = data.query('extra_runs == 0 & over >= 16')
Bowler_death_over= Bowler_death_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_death_over["Death_over"]=np.round(Bowler_death_over["ball"]/6,1)

Bowler_death = pd.merge(Bowler_death_wickets,Bowler_death_runs,on="bowler")
Bowler_death = pd.merge(Bowler_death, Bowler_death_over,on="bowler")
Bowler_death["Death_economy"] = np.round(Bowler_death["Death_runs"] / Bowler_death["Death_over"],1)
del Bowler_death['Death_wickets']
del Bowler_death['ball']

Bowler_death.head(5)

,bowler,Death_runs,Death_over,Death_economy
0,A Ashish Reddy,123,11.5,10.7
1,A Chandila,6,1.0,6.0
2,A Choudhary,40,4.0,10.0
3,A Flintoff,52,3.0,17.3
4,A Kumble,148,17.0,8.7


<b>calculating number of times bowler has taken 4 wickets</b>

In [27]:
data_without_runouts = data[data["dismissal_kind"] != "run out"]
no_of_4_wickets=data_without_runouts.groupby(['bowler','id'])['is_wicket'].agg(sum).reset_index(name="wickets")
no_of_4_wickets.head(5)

,bowler,id,wickets
0,A Ashish Reddy,548329,0
1,A Ashish Reddy,548341,2
2,A Ashish Reddy,548346,1
3,A Ashish Reddy,548348,1
4,A Ashish Reddy,548352,1


In [28]:
no_of_4_wickets=no_of_4_wickets[no_of_4_wickets['wickets']>=4]
no_of_4_wickets=no_of_4_wickets.groupby('bowler').size().reset_index(name='4W')
no_of_4_wickets.head(5)

,bowler,4W
0,A Chandila,1
1,A Kumble,3
2,A Mishra,4
3,A Nehra,1
4,A Singh,1


In [29]:
#Merging the data
Bowler_details = Bowler_over.merge(Bowler_Runs,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_wicket,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_matches,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_boundary,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_Dots,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_powerplay,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_middle,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_death,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(no_of_4_wickets,left_on='bowler',right_on='bowler',how='left')
Bowler_details.head(5)

,bowler,ball,No_of_Overs,Total_runs_given,Wickets,no_of_match,Total_boundaries_given,Dot_balls,Powerplay_wickets,Powerplay_runs,Powerplay_over,Powerplay_economy,Middle_wickets,Middle_runs,Middle_over,Middle_economy,Death_runs,Death_over,Death_economy,4W
0,Harbhajan Singh,3325,554.0,4038,150,157,431,1244,30.0,894.0,122.0,7.3,101.0,2904.0,404.0,7.2,240.0,28.2,8.5,2.0
1,PP Chawla,3218,536.0,4330,156,163,497,1137,18.0,445.0,58.5,7.6,112.0,3463.0,436.2,7.9,422.0,41.7,10.1,2.0
2,R Ashwin,3186,531.0,3756,138,151,361,1166,42.0,1003.0,150.8,6.7,83.0,2384.0,338.0,7.1,369.0,42.2,8.7,1.0
3,A Mishra,3128,521.0,3913,160,150,394,1125,6.0,164.0,21.7,7.6,130.0,3319.0,457.3,7.3,430.0,42.3,10.2,4.0
4,SL Malinga,2753,458.0,3486,170,122,400,1144,37.0,1210.0,180.7,6.7,43.0,1024.0,131.2,7.8,1252.0,147.0,8.5,7.0


<b>Calculating Economy,Strike Rate,Avg. wickets,% dot balls,% bounderies,bowling avg</b>

In [30]:
Bowler_details["Economy"] = np.round(Bowler_details["Total_runs_given"]/Bowler_details["No_of_Overs"],1)
Bowler_details["Strike Rate"] = np.round(Bowler_details["ball"]/Bowler_details["Wickets"],1)
Bowler_details["Avg. Wickets"] = np.round(Bowler_details["Wickets"]/Bowler_details["no_of_match"],1)
Bowler_details["%age of dots"] = np.round((Bowler_details["Dot_balls"]/Bowler_details["ball"]) * 100,1)
Bowler_details["%boundaries"] = np.round((Bowler_details["Total_boundaries_given"]/Bowler_details["ball"]) * 100,1)
Bowler_details["bowl_avg"] = np.round((Bowler_details["Total_runs_given"]/Bowler_details["Wickets"]),1)
Bowler_details.fillna(0.0)
Bowler_details.replace(np.nan, 0)
Bowler_details.to_csv('bowler.csv')
Bowler_details.head(5)

,bowler,ball,No_of_Overs,Total_runs_given,Wickets,no_of_match,Total_boundaries_given,Dot_balls,Powerplay_wickets,Powerplay_runs,...,Death_runs,Death_over,Death_economy,4W,Economy,Strike Rate,Avg. Wickets,%age of dots,%boundaries,bowl_avg
0,Harbhajan Singh,3325,554.0,4038,150,157,431,1244,30.0,894.0,...,240.0,28.2,8.5,2.0,7.3,22.2,1.0,37.4,13.0,26.9
1,PP Chawla,3218,536.0,4330,156,163,497,1137,18.0,445.0,...,422.0,41.7,10.1,2.0,8.1,20.6,1.0,35.3,15.4,27.8
2,R Ashwin,3186,531.0,3756,138,151,361,1166,42.0,1003.0,...,369.0,42.2,8.7,1.0,7.1,23.1,0.9,36.6,11.3,27.2
3,A Mishra,3128,521.0,3913,160,150,394,1125,6.0,164.0,...,430.0,42.3,10.2,4.0,7.5,19.6,1.1,36.0,12.6,24.5
4,SL Malinga,2753,458.0,3486,170,122,400,1144,37.0,1210.0,...,1252.0,147.0,8.5,7.0,7.6,16.2,1.4,41.6,14.5,20.5


In [31]:
#Rank calculation
Bowler_details['runs_given_rank']=Bowler_details['Total_runs_given'].rank(ascending=True)
Bowler_details['bowl_avg_rank']=Bowler_details['bowl_avg'].rank(ascending=True)
Bowler_details['experience_rank']=Bowler_details['No_of_Overs'].rank(ascending=False)
Bowler_details['wickets_rank']=Bowler_details['Wickets'].rank(ascending=False)
Bowler_details['dot_balls_rank']=Bowler_details['%age of dots'].rank(ascending=False)
Bowler_details['boundary_ball_rank']=Bowler_details['%boundaries'].rank(ascending=True)
Bowler_details['economy_rank']=Bowler_details['Economy'].rank(ascending=True)
Bowler_details['powerplay_eco_rank']=Bowler_details['Powerplay_economy'].rank(ascending=True)
Bowler_details['deathover_eco_rank']=Bowler_details['Death_economy'].rank(ascending=True)
BowlerRank=Bowler_details[['bowler','runs_given_rank','bowl_avg_rank','experience_rank','wickets_rank','dot_balls_rank','boundary_ball_rank'
                          ,'economy_rank','powerplay_eco_rank','deathover_eco_rank']]
BowlerRank['bowling_rank']=BowlerRank[['bowl_avg_rank','experience_rank','wickets_rank','dot_balls_rank','boundary_ball_rank'
                          ,'economy_rank']].mean(axis=1)
BowlerRank['bowling_rank']=np.round(BowlerRank.bowling_rank.rank(ascending=True),0)
BowlerRank['powerplay_rank']=BowlerRank[['powerplay_eco_rank','dot_balls_rank','wickets_rank']].mean(axis=1)
BowlerRank['powerplay_rank']=BowlerRank.powerplay_rank.rank(ascending=True)
BowlerRank['deathover_rank']=BowlerRank[['deathover_eco_rank','boundary_ball_rank']].mean(axis=1)
BowlerRank['deathover_rank']=BowlerRank.deathover_rank.rank(ascending=True)
BowlerRank

C:\Users\Admin\AppData\Local\Temp/ipykernel_15080/1175521295.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BowlerRank['bowling_rank']=BowlerRank[['bowl_avg_rank','experience_rank','wickets_rank','dot_balls_rank','boundary_ball_rank'
C:\Users\Admin\AppData\Local\Temp/ipykernel_15080/1175521295.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BowlerRank['bowling_rank']=np.round(BowlerRank.bowling_rank.rank(ascending=True),0)
C:\Users\Admin\AppData\Local\Temp/ipykernel_15080/1175521295.py:16: Settin

,bowler,runs_given_rank,bowl_avg_rank,experience_rank,wickets_rank,dot_balls_rank,boundary_ball_rank,economy_rank,powerplay_eco_rank,deathover_eco_rank,bowling_rank,powerplay_rank,deathover_rank
0,Harbhajan Singh,297.0,80.5,1.0,5.0,112.5,42.0,27.5,70.0,29.5,9.0,30.0,22.0
1,PP Chawla,298.0,98.5,2.0,3.0,153.0,94.0,93.5,101.0,109.5,29.0,59.0,92.0
2,R Ashwin,294.0,87.5,3.0,6.0,126.5,10.0,14.5,39.0,36.5,5.0,24.0,12.0
3,A Mishra,296.0,56.0,4.0,2.0,137.5,32.0,36.0,101.0,115.0,8.0,49.5,57.0
4,SL Malinga,291.0,16.5,5.0,1.0,57.0,74.0,45.0,39.0,29.5,3.0,4.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,A Flintoff,10.0,274.0,295.0,288.5,237.5,250.0,281.0,242.0,265.0,292.0,278.0,280.0
294,DNT Zoysa,8.0,267.0,295.0,288.5,121.0,273.0,261.5,89.5,207.0,282.0,190.0,266.0
295,MJ Clarke,1.0,209.5,295.0,288.5,168.0,3.0,20.5,NaN,NaN,178.0,255.0,1.0
296,S Dube,2.0,20.0,295.0,268.0,286.0,35.0,116.0,NaN,73.5,191.0,292.0,40.0


In [32]:
#Calculating all rounder rank

r1=batsmanRank[['batsman','batsman_rank']]
r1=r1.rename(columns={'batsman':'Player'})
r2=BowlerRank[['bowler','bowling_rank']]
r2=r2.rename(columns={'bowler':'Player'})
AllRank=pd.merge(r1,r2,how='outer',on='Player')
AllRank['allrounder_rank']=(AllRank['batsman_rank']+AllRank['bowling_rank'])/2
AllRank['allrounder_rank']=np.round(AllRank['allrounder_rank'].rank(ascending=True),0)
AllRank.replace(np.nan,"-",inplace=True)

AllRank

,Player,batsman_rank,bowling_rank,allrounder_rank
0,V Sehwag,8.0,278.0,68.0
1,BB McCullum,16.0,-,-
2,DA Warner,1.0,-,-
3,RV Uthappa,20.0,-,-
4,AC Gilchrist,24.0,-,-
...,...,...,...,...
412,AP Dole,-,264.0,-
413,A Flintoff,-,292.0,-
414,DNT Zoysa,-,282.0,-
415,MJ Clarke,-,178.0,-


# Team KPIs

1. Percentage wins
2. Percentage wins by chasing
3. Percentage wins by defending
4. Percentage wins by tie
5. Percentage toss wins
6. Percentage home wins
7. Number of times qualified
8. Number of times toss and match winner

<b>Calculating number of matches for each team</b>

In [33]:
no_of_match_as_team1=df.groupby('team1').size().reset_index(name="no_of_match_as_team1")
no_of_match_as_team1=no_of_match_as_team1.rename(columns={"team1":"team_name"})
no_of_match_as_team1.head(5)

,team_name,no_of_match_as_team1
0,Chennai Super Kings,94
1,Delhi Capitals,102
2,Gujarat Lions,16
3,Kings XI Punjab,92
4,Kochi Tuskers Kerala,7


In [34]:
no_of_match_as_team2=df.groupby('team2').size().reset_index(name="no_of_match_as_team2")
no_of_match_as_team2=no_of_match_as_team2.rename(columns={"team2":"team_name"})
no_of_match_as_team2.head(5)

,team_name,no_of_match_as_team2
0,Chennai Super Kings,84
1,Delhi Capitals,92
2,Gujarat Lions,14
3,Kings XI Punjab,98
4,Kochi Tuskers Kerala,7


In [35]:
no_of_matches=pd.merge(no_of_match_as_team1,no_of_match_as_team2,on="team_name")
no_of_matches['total_no_of_matches_played']=no_of_matches['no_of_match_as_team1']+no_of_matches['no_of_match_as_team2']
no_of_matches.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played
0,Chennai Super Kings,94,84,178
1,Delhi Capitals,102,92,194
2,Gujarat Lions,16,14,30
3,Kings XI Punjab,92,98,190
4,Kochi Tuskers Kerala,7,7,14


<b>Insert the total number of wins for each teams</b>

In [36]:
no_of_wins=df.groupby('winner').size().reset_index(name="no_of_wins")
no_of_wins=no_of_wins.rename(columns={'winner':'team_name'})
no_of_wins.head(5)

,team_name,no_of_wins
0,Chennai Super Kings,106
1,Delhi Capitals,86
2,Gujarat Lions,13
3,Kings XI Punjab,88
4,Kochi Tuskers Kerala,6


In [37]:
#merging
teams_df=pd.merge(no_of_matches,no_of_wins,on='team_name')
teams_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins
0,Chennai Super Kings,94,84,178,106
1,Delhi Capitals,102,92,194,86
2,Gujarat Lions,16,14,30,13
3,Kings XI Punjab,92,98,190,88
4,Kochi Tuskers Kerala,7,7,14,6


<b>Calculating the number of wins by defending</b>

In [38]:
win_by_defending=df[df['result']=='runs'].groupby('winner').size().reset_index(name="wins_defend")
win_by_defending=win_by_defending.rename(columns={'winner':'team_name'})
win_by_defending.head(5)

,team_name,wins_defend
0,Chennai Super Kings,53
1,Delhi Capitals,34
2,Gujarat Lions,1
3,Kings XI Punjab,40
4,Kochi Tuskers Kerala,2


<b>Calculating total number of wins by chase</b>

In [39]:
win_by_chase=df[df['result']=='wickets'].groupby('winner').size().reset_index(name="wins_chase")
win_by_chase=win_by_chase.rename(columns={'winner':'team_name'})
win_by_chase.head(5)

,team_name,wins_chase
0,Chennai Super Kings,53
1,Delhi Capitals,50
2,Gujarat Lions,12
3,Kings XI Punjab,45
4,Kochi Tuskers Kerala,4


<b>Calculating toss wins for each team</b>

In [40]:
toss_wins=df.groupby('toss_winner').size().reset_index(name="toss_wins")
toss_wins=toss_wins.rename(columns={'toss_winner':'team_name'})
toss_wins.head(5)

,team_name,toss_wins
0,Chennai Super Kings,97
1,Delhi Capitals,100
2,Gujarat Lions,15
3,Kings XI Punjab,85
4,Kochi Tuskers Kerala,8


In [41]:
win_by_tie=df[df['result']=='tie'].groupby('winner').size().reset_index(name="wins_tie")
win_by_tie=win_by_tie.rename(columns={'winner':'team_name'})
win_by_tie.head(5)

,team_name,wins_tie
0,Delhi Capitals,2
1,Kings XI Punjab,3
2,Kolkata Knight Riders,1
3,Mumbai Indians,2
4,Rajasthan Royals,2


In [42]:
#merging
team_df = pd.merge(teams_df,win_by_chase,on="team_name")
team_df = pd.merge(team_df,win_by_defending,on="team_name")
team_df = team_df.merge(win_by_tie, left_on='team_name', right_on='team_name', how='left')
team_df = pd.merge(team_df, toss_wins, on ='team_name')
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97
1,Delhi Capitals,102,92,194,86,50,34,2.0,100
2,Gujarat Lions,16,14,30,13,12,1,NaN,15
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8


<b>Calculating percentage wins, percentage wins by chase,percentage wins by defend,percentage wins by tie and percentage toss</b>

In [43]:
teams_df['%win']=np.round((teams_df['no_of_wins']/teams_df['total_no_of_matches_played'])*100,1)
team_df['%win_chase']=np.round((team_df['wins_chase']/team_df['no_of_wins'])*100,2)
team_df['%win_defend']=np.round((team_df['wins_defend']/team_df['no_of_wins'])*100,2)
team_df['%win_tie']=np.round((team_df['wins_tie']/team_df['no_of_wins'])*100,1)
team_df['%toss_wins']=np.round((team_df['toss_wins']/team_df['total_no_of_matches_played'])*100,1)
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1


<b>Calculating Percentage win in homeground for each time</b>

In [44]:
data_citiy=pd.DataFrame(df.groupby('city'))
homegrounds={'Kolkata':'Kolkata Knight Riders','Bangalore':'Royal Challengers Bangalore','Bengaluru':'Royal Challengers Bangalore',
            'Chandigarh':'Kings XI Punjab','Chennai':'Chennai Super Kings','Delhi':'Delhi Capitals','Hyderabad':'Sunrisers Hyderabad',
            'Jaipur':'Rajasthan Royals','Rajkot':'Gujarat Lions','Kochi':'Kochi Tuskers Kerala','Kanpur':'Gujarat Lions',
            'Visakhapatnam':'Rising Pune Supergiants','Pune':'Rising Pune Supergiants','Cuttack':'Sunrisers Hyderabad',
            'Dharamshala':'Kings XI Punjab','Ahmedabad':'Rajasthan Royals','Mohali':'Kings XI Punjab','Mumbai':'Mumbai Indians'}

In [45]:
data_without_neutral=df[df['neutral_venue']!=1]
data_without_neutral=data_without_neutral[data_without_neutral['city']!='Dubai']
data_without_neutral['home_venue']=data_without_neutral['city'].map(homegrounds)
data_without_neutral.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,home_venue
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,Royal Challengers Bangalore
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,Kings XI Punjab
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi Capitals
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,Mumbai Indians
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,Kolkata Knight Riders


In [46]:
data_without_neutral['win']=np.where(data_without_neutral['winner'] == data_without_neutral['home_venue'], True, False)
data_without_neutral.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,home_venue,win
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,Royal Challengers Bangalore,False
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,Kings XI Punjab,False
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi Capitals,True
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,Mumbai Indians,False
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,Kolkata Knight Riders,True


In [47]:
no_of_wins_homeground=data_without_neutral[data_without_neutral['win']==True]
no_of_wins_homeground_per_team=no_of_wins_homeground.groupby('winner').size().reset_index(name="home_wins_count")
no_of_wins_homeground_per_team=no_of_wins_homeground_per_team.rename(columns={'winner':'team_name'})
no_of_wins_homeground_per_team.head(5)

,team_name,home_wins_count
0,Chennai Super Kings,40
1,Delhi Capitals,31
2,Gujarat Lions,5
3,Kings XI Punjab,30
4,Kochi Tuskers Kerala,2


In [48]:
total_home_matches_per_team=data_without_neutral.groupby('home_venue').size().reset_index(name="home_match_count")
total_home_matches_per_team=total_home_matches_per_team.rename(columns={'home_venue':'team_name'})

In [49]:
total_home_matches_per_team.head(5)

,team_name,home_match_count
0,Chennai Super Kings,57
1,Delhi Capitals,74
2,Gujarat Lions,14
3,Kings XI Punjab,56
4,Kochi Tuskers Kerala,5


In [50]:
home_match_data=total_home_matches_per_team.merge(no_of_wins_homeground_per_team,left_on='team_name',right_on='team_name', how='left')
home_match_data.head(5)

,team_name,home_match_count,home_wins_count
0,Chennai Super Kings,57,40
1,Delhi Capitals,74,31
2,Gujarat Lions,14,5
3,Kings XI Punjab,56,30
4,Kochi Tuskers Kerala,5,2


In [51]:
home_match_data['%home_win']=np.round((home_match_data['home_wins_count']/home_match_data['home_match_count'])*100,2)
home_match_data.head(5)

,team_name,home_match_count,home_wins_count,%home_win
0,Chennai Super Kings,57,40,70.18
1,Delhi Capitals,74,31,41.89
2,Gujarat Lions,14,5,35.71
3,Kings XI Punjab,56,30,53.57
4,Kochi Tuskers Kerala,5,2,40.00


In [52]:
del home_match_data['home_match_count']
del home_match_data['home_wins_count']
home_match_data.head(5)

,team_name,%home_win
0,Chennai Super Kings,70.18
1,Delhi Capitals,41.89
2,Gujarat Lions,35.71
3,Kings XI Punjab,53.57
4,Kochi Tuskers Kerala,40.00


In [53]:
team_df=team_df.merge(home_match_data, left_on='team_name', right_on='team_name', how='left')
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00


<b>Calculating the total number of times each team qualified</b>

In [54]:
#added year column in main data
df['year']=df['date'].map(lambda x:x[0:4])
df.head(5)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,year
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,2008
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,2008
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,2008
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,2008


In [55]:
year_wise_match_team1=df.groupby(['year','team1']).size().reset_index(name="no_of_match_team1")
year_wise_match_team1=year_wise_match_team1.rename(columns={'team1':'team_name'})
year_wise_match_team1.head(5)

,year,team_name,no_of_match_team1
0,2008,Chennai Super Kings,9
1,2008,Delhi Capitals,7
2,2008,Kings XI Punjab,7
3,2008,Kolkata Knight Riders,7
4,2008,Mumbai Indians,7


In [56]:
year_wise_match_team2=df.groupby(['year','team2']).size().reset_index(name="no_of_match_team2")
year_wise_match_team2=year_wise_match_team2.rename(columns={'team2':'team_name'})
year_wise_match_team2.head(5)

,year,team_name,no_of_match_team2
0,2008,Chennai Super Kings,7
1,2008,Delhi Capitals,7
2,2008,Kings XI Punjab,8
3,2008,Kolkata Knight Riders,6
4,2008,Mumbai Indians,7


In [57]:
year_wise_matches=pd.merge(year_wise_match_team1,year_wise_match_team2,on=['year','team_name'])
year_wise_matches['total_matches']=year_wise_matches['no_of_match_team1']+year_wise_matches['no_of_match_team2']
year_wise_matches.head(5)

,year,team_name,no_of_match_team1,no_of_match_team2,total_matches
0,2008,Chennai Super Kings,9,7,16
1,2008,Delhi Capitals,7,7,14
2,2008,Kings XI Punjab,7,8,15
3,2008,Kolkata Knight Riders,7,6,13
4,2008,Mumbai Indians,7,7,14


In [58]:
year_wise_qualified=year_wise_matches[year_wise_matches['total_matches']>14]
year_wise_qualified.head(5)

,year,team_name,no_of_match_team1,no_of_match_team2,total_matches
0,2008,Chennai Super Kings,9,7,16
2,2008,Kings XI Punjab,7,8,15
5,2008,Rajasthan Royals,7,9,16
9,2009,Delhi Capitals,9,6,15
13,2009,Sunrisers Hyderabad,10,6,16


In [59]:
no_of_time_team_qualified=year_wise_qualified.groupby('team_name').size().reset_index(name='toatal_no_of_time_qualified')
no_of_time_team_qualified.head(5)

,team_name,toatal_no_of_time_qualified
0,Chennai Super Kings,9
1,Delhi Capitals,5
2,Gujarat Lions,1
3,Kings XI Punjab,4
4,Kolkata Knight Riders,7


In [60]:
team_df=team_df.merge(no_of_time_team_qualified,left_on='team_name',right_on='team_name',how='left')
team_df.head(5)

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win,toatal_no_of_time_qualified
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18,9.0
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89,5.0
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71,1.0
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57,4.0
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00,NaN


In [61]:
toss_match_winner=df[df['toss_winner']==df['winner']]
toss_match_winner=toss_match_winner.groupby('winner').size().reset_index(name='toss_match_winner')
team_df=pd.merge(team_df,toss_match_winner,left_on='team_name',right_on='winner',how='left')
team_df['% win']=np.round((team_df['no_of_wins']/team_df['total_no_of_matches_played'])*100,2)
team_df

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win,toatal_no_of_time_qualified,winner,toss_match_winner,% win
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18,9.0,Chennai Super Kings,61,59.55
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89,5.0,Delhi Capitals,45,44.33
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71,1.0,Gujarat Lions,10,43.33
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57,4.0,Kings XI Punjab,36,46.32
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00,NaN,Kochi Tuskers Kerala,4,42.86
5,Kolkata Knight Riders,95,97,192,99,57,41,1.0,98,57.58,41.41,1.0,51.0,58.44,7.0,Kolkata Knight Riders,55,51.56
6,Mumbai Indians,97,106,203,120,56,62,2.0,106,46.67,51.67,1.7,52.2,52.48,9.0,Mumbai Indians,61,59.11
7,Rajasthan Royals,70,91,161,81,51,28,2.0,87,62.96,34.57,2.5,54.0,66.10,4.0,Rajasthan Royals,44,50.31
8,Rising Pune Supergiants,37,39,76,27,14,13,NaN,33,51.85,48.15,NaN,43.4,21.57,3.0,Rising Pune Supergiants,11,35.53
9,Royal Challengers Bangalore,108,87,195,91,51,38,2.0,87,56.04,41.76,2.2,44.6,46.25,7.0,Royal Challengers Bangalore,43,46.67


# Match KPIs

1. Percentage wins by chasing
2. Percentage wins by defending
3. Perceptage of tie matches
4. Percentage won the toss as well as match

<b>Removing matches tied</b>

In [62]:
#Excluding tie matches
total_matches = df[df['result'] != 'tie']
total_matches["result"].count()

799

<b>Wins by defending</b>

In [63]:
run_wins = total_matches[total_matches['result'] == 'runs']
run_wins['result'].count()

364

<b>Wins by chasing</b>

In [64]:
wicket_wins = total_matches[total_matches['result'] == 'wickets']
wicket_wins['result'].count()

435

<b>Percentage wins by chasing runs</b>

In [65]:
#Percentage wins by chasing runs
np.round((run_wins['result'].count()/total_matches["result"].count())*100,1)

45.6

<b>Percentage wins by defending</b>

In [66]:
#Percentage wins by defending
np.round((wicket_wins['result'].count()/total_matches["result"].count())*100,1)

54.4

<b>Perceptage of tie matches</b>

In [67]:
#Perceptage of tie matches
tie_matches = df[df['result'] == 'tie']
np.round(tie_matches['result'].count()/df["result"].count()*100,1)

1.6

<b>Percentage won the toss as well as match</b>

In [68]:
#Percentage won the toss as well as match
toss_winner = df[df['toss_winner'] == df['winner']]
np.round(toss_winner['result'].count()/df["result"].count()*100,1)

51.5

In [69]:
mergedTable = data.merge(df, left_on='id',right_on='id',how='left')
mergedTable.head(10)

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,...,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,year
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
5,335982,1,7,4,BB McCullum,RT Ponting,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
6,335982,1,7,5,RT Ponting,BB McCullum,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
7,335982,1,7,6,BB McCullum,RT Ponting,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
8,335982,1,8,1,BB McCullum,RT Ponting,JH Kallis,0,0,0,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
9,335982,1,8,2,BB McCullum,RT Ponting,JH Kallis,0,0,0,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008


In [70]:
#Exporting
Batsman_Df=pd.merge(batsmanDf,AllRank,left_on='batsman',right_on='Player',how='left')
Batsman_Df.to_csv('batsman.csv')
Bowler_Df=pd.merge(Bowler_details,AllRank,left_on='bowler',right_on='Player',how='left')
Bowler_Df.to_csv('bowler.csv')